In [37]:
import torch
import torch.nn as nn
import torch.nn.functional as F #torch是关于运算的包
import torchvision
from torchvision import datasets,transforms, models #torchvision则是打包了一些数据集
from torch.utils.data import Dataset
from torchvision import transforms as T
import pandas as pd
import numpy as np
import glob
from natsort import natsorted
from PIL import Image


In [24]:
label2int = {'airplane':0, 'automobile':1, 'bird':2, 'cat':3, 'deer':4, 'dog':5, 'frog':6, 'horse':7, 'ship':8, 'truck':9}
int2lable = {0:'airplane', 1:'automobile', 2:'bird', 3:'cat', 4:'deer', 5:'dog', 6:'frog', 7:'horse', 8:'ship', 9:'truck'}

In [42]:
class TrainAndValData(Dataset):
    def __init__(self, img_path, csv_path, train=True, transforms=None):
        '''
        获得所有图片路径，并划分训练集、验证集
        '''
        self.train = train
        files = natsorted(glob.glob(img_path + '/*'))
        files_num = len(files)
        if self.train:
            self.img_name = files[: int(0.9*files_num)]
        else:
            self.img_name = files[int(0.9*files_num): ]
        self.img_label = pd.read_csv(csv_path).values[:, 1]
        
        #数据增强
        if transforms is None:
            normalize = T.Normalize(mean=[0.485, 0.0456, 0.406], std=[0.229, 0.224, 0.225])
            #训练集用数据增强
            if self.train:
                self.transforms = T.Compose([
                    T.Scale(256),
                    T.RandomSizedCrop(224),
                    T.RandomHorizontalFlip(),
                    T.ToTensor(),
                    normalize 
                ])
            else:
                self.transforms = T.Compose([
                    T.Scale(224),
                    T.CenterCrop(224),
                    T.ToTensor(),
                    normalize 
                ])
    def __len__(self):
        '''
        返回数据集中所有图片的个数
        '''
        return len(self.img_name)
    def __getitem__(self, index):
        '''
        返回一张图片的数据
        '''
        img_path = self.img_name[index]
        img = Image.open(img_path)
        img = self.transforms(img)
        label = label2int[self.img_label[index]]
        return img, label

In [43]:
img_path = 'data/train'
csv_path = 'data/trainLabels.csv'
train_dataset = TrainAndValData(img_path, csv_path, train=True)
test_dataset = TrainAndValData(img_path, csv_path, train=False)

/home/lhw/anaconda3/lib/python3.6/site-packages/torchvision/transforms/transforms.py:188: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")
/home/lhw/anaconda3/lib/python3.6/site-packages/torchvision/transforms/transforms.py:563: UserWarning: The use of the transforms.RandomSizedCrop transform is deprecated, please use transforms.RandomResizedCrop instead.
  "please use transforms.RandomResizedCrop instead.")


In [45]:
#print(train_dataset[0][0])打印图片数据
print(train_dataset[0][1])#打印标签
print(len(train_dataset))

6
45000


In [46]:
### 定义网络 ###
cfg = {
    'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}

In [47]:
class VGG(nn.Module):
    def __init__(self, vgg_name):
        super(VGG, self).__init__()
        self.features = self._make_layers(cfg[vgg_name])
        self.classifier = nn.Linear(512, 10)

    def forward(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return out

    def _make_layers(self, cfg):
        layers = []
        in_channels = 3
        for x in cfg:
            if x == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                           nn.BatchNorm2d(x),
                           nn.ReLU(inplace=True)]
                in_channels = x
        layers += [nn.AvgPool2d(kernel_size=1, stride=1)]
        return nn.Sequential(*layers)

In [48]:
net = VGG('VGG16')
#print(net)
x = torch.randn(10,3,32,32)
y = net(x)